In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
import numpy as np
from torch.utils.data import DataLoader, TensorDataset


In [4]:
class MinesweeperBoard:
    def __init__(self, width, height, mines):
        self.width = width
        self.height = height
        self.mines = set(mines)
        self.revealed = set()
        self.flagged = set()
    
    def in_bounds(self, x, y):
        return 0 <= x < self.width and 0 <= y < self.height
    
    def adjacent_cells(self, x, y):
        return [(x + dx, y + dy) for dx in [-1, 0, 1] for dy in [-1, 0, 1] if (dx, dy) != (0, 0) and self.in_bounds(x + dx, y + dy)]

    def count_adjacent_mines(self, x, y):
        return sum(1 for nx, ny in self.adjacent_cells(x, y) if (nx, ny) in self.mines)
    
    def reveal(self, x, y):
        if (x, y) in self.revealed:
            return []
        
        self.revealed.add((x, y))
        
        if (x, y) in self.mines:
            return [(x, y, "mine")]

        count = self.count_adjacent_mines(x, y)
        if count == 0:
            return [(x, y, count)] + [cell for nx, ny in self.adjacent_cells(x, y) for cell in self.reveal(nx, ny)]
        else:
            return [(x, y, count)]

    def is_game_over(self):
        return len(self.revealed) == self.width * self.height - len(self.mines)

In [28]:
class MinesweeperCNN(nn.Module):
    def __init__(self):
        super(MinesweeperCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv3 = nn.Conv2d(32, 32, 3, padding=1)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1)
        self.fc = nn.Linear(32 * 30 * 16, 1)
        
    def forward(self, x):
        x = nn.ReLU()(self.conv1(x))
        x = nn.ReLU()(self.conv2(x))
        x = nn.ReLU()(self.conv3(x))
        x = nn.ReLU()(self.conv4(x))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return torch.sigmoid(x)

def generate_random_board(width, height, num_mines):
    board = np.zeros((height, width), dtype=int)
    mine_indices = np.random.choice(width * height, num_mines, replace=False)
    for idx in mine_indices:
        row, col = divmod(idx, width)
        board[row, col] = 1
    return board

def generate_training_data(board, num_samples):
    width, height = board.shape
    X = np.zeros((num_samples, 1, height, width), dtype=np.float32)
    Y = np.zeros((num_samples, 1), dtype=np.float32)

    for i in range(num_samples):
        sample_board = np.copy(board)
        np.random.shuffle(sample_board.ravel())
        sample_board = sample_board.reshape(height, width)
        X[i, 0] = sample_board
        Y[i, 0] = float(np.sum(sample_board))

    return TensorDataset(torch.from_numpy(X), torch.from_numpy(Y))

def train(model, board, epochs, lr, batch_size=64):
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        epoch_loss = 0.0
        num_batches = 0

        for _ in range(batch_size):
            sample_board = generate_random_board(*board.shape, board.sum())
            data = generate_training_data(sample_board, 1)
            data_loader = DataLoader(data, batch_size=1)

            for batch_X, batch_Y in data_loader:
                optimizer.zero_grad()
                outputs = model(batch_X)
                loss = criterion(outputs, batch_Y)
                loss.backward()
                optimizer.step()

                epoch_loss += loss.item()
                num_batches += 1

        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/num_batches}")



In [29]:
width, height = 30, 16
num_mines = 99
epochs = 40
lr = 0.001

board = generate_random_board(width, height, num_mines)
model = MinesweeperCNN()
train(model, board, epochs, lr)
model_path = "minesweeper_cnn_model.pt"
torch.save(model.state_dict(), model_path)


[W NNPACK.cpp:64] Could not initialize NNPACK! Reason: Unsupported hardware.


Epoch 1/40, Loss: 9608.77734375
Epoch 2/40, Loss: 9604.0
Epoch 3/40, Loss: 9604.0
Epoch 4/40, Loss: 9604.0
Epoch 5/40, Loss: 9604.0
Epoch 6/40, Loss: 9604.0
Epoch 7/40, Loss: 9604.0
Epoch 8/40, Loss: 9604.0
Epoch 9/40, Loss: 9604.0
Epoch 10/40, Loss: 9604.0
Epoch 11/40, Loss: 9604.0
Epoch 12/40, Loss: 9604.0
Epoch 13/40, Loss: 9604.0
Epoch 14/40, Loss: 9604.0
Epoch 15/40, Loss: 9604.0
Epoch 16/40, Loss: 9604.0
Epoch 17/40, Loss: 9604.0
Epoch 18/40, Loss: 9604.0
Epoch 19/40, Loss: 9604.0
Epoch 20/40, Loss: 9604.0
Epoch 21/40, Loss: 9604.0
Epoch 22/40, Loss: 9604.0
Epoch 23/40, Loss: 9604.0
Epoch 24/40, Loss: 9604.0
Epoch 25/40, Loss: 9604.0
Epoch 26/40, Loss: 9604.0
Epoch 27/40, Loss: 9604.0
Epoch 28/40, Loss: 9604.0
Epoch 29/40, Loss: 9604.0
Epoch 30/40, Loss: 9604.0
Epoch 31/40, Loss: 9604.0
Epoch 32/40, Loss: 9604.0
Epoch 33/40, Loss: 9604.0
Epoch 34/40, Loss: 9604.0
Epoch 35/40, Loss: 9604.0
Epoch 36/40, Loss: 9604.0
Epoch 37/40, Loss: 9604.0
Epoch 38/40, Loss: 9604.0
Epoch 39/40, L